In [56]:
from selenium import webdriver
from sqlalchemy import true
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import time
import random
from selenium.webdriver.common.by import By

# Initiate the browser
browser  = webdriver.Chrome(ChromeDriverManager().install())

# Open the Website
browser.get('https://www.linkedin.com/login/es?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Login info
linkedin_username = 'vallarinopedro@gmail.com'
#linkedin_password = 

#Input password
browser.find_element(By.XPATH,'//*[@id="username"]').send_keys(linkedin_username)
browser.find_element(By.XPATH,'//*[@id="password"]').send_keys(linkedin_password)
browser.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()

In [57]:
linkedin_job_links =   [
    'https://www.linkedin.com/jobs/search/?currentJobId=3333640239&distance=25&f_C=3110591&f_E=1%2C2&f_JT=F&f_TPR=r86400&f_WT=2&geoId=92000000&keywords=Product%20Analyst&sortBy=R',
    'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3331004870&f_TPR=a1666888102-&keywords=Product%20Analyst&savedSearchId=1730281946&searchAlertRefId=3a43863e-8389-4dc5-99d2-a59556bf977d',
    'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3331004894&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1666825009-&f_WT=2&keywords=Data%20Analyst&savedSearchId=1730274642&searchAlertRefId=2fcc8f37-361a-4502-92db-3a09d4497f67',
    'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3333154988&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1666825009-&f_WT=2&keywords=Product%20Owner&savedSearchId=1730274666&searchAlertRefId=45369823-0864-4152-bd8c-48ed48861cde',
    'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3322491585&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1666825009-&f_WT=2&keywords=Data%20Scientist&savedSearchId=1730274650&searchAlertRefId=bc1b5913-a8d4-43e9-81bd-c7fa461f90de'
                        ]        

linkedin_csv_headers = ['Test','Product Analyst','Data Analyst','Product Owner','Data Scientist']

# Upsizing of the windows and setting window position
browser.set_window_position(1, 1)
browser.maximize_window()

contador_global = 0

for link in linkedin_job_links:
    df = pd.DataFrame(columns=['Job Title','Job Company','Job Location','Link','Job Description','Job Words'])
    browser.get(link)

    number_of_jobs = re.findall(r'\d+', browser.find_element(By.XPATH,'//*[@id="main"]/div/section[1]/header/div[1]/small').get_attribute('innerText'))
    number_of_jobs = ''.join(number_of_jobs)
    number_of_jobs = int(number_of_jobs)
    print('The number of jobs of this search is',number_of_jobs)

    # For DOM load, 1 second wait time is appropiate
    time.sleep(2)

    while True:        
        pagination_elements = browser.find_elements(By.XPATH,"//li[contains(@class,'artdeco-pagination__indicator artdeco-pagination__indicator--number')]")
        page_number_array = []

        for element in pagination_elements:
            page_number_array.append(int(re.findall(r'\d+',element.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]))

        try:
            if browser.find_element(By.XPATH,"//h1[contains(@class,'t-24 t-black t-normal text-align-center')]").get_attribute('innerText') == 'No se han encontrado empleos para esta búsqueda.':
                df = df.reset_index(drop = True)
                df.to_excel(str(linkedin_csv_headers[contador_global]) + '.xlsx')
                break
        except:
            pass

        # For DOM load, 1 second wait time is appropiate
        time.sleep(1)

        # All job cards are loaded to then be reference points to scroll by
        jobs = browser.find_element(By.XPATH,"//ul[contains(@class,'scaffold-layout__list-container')]").find_elements_by_xpath('./*')

        # Due to lazy loading scroll down is necessary to load all the neccesary information in the HTML 
        for job in jobs:
            job.location_once_scrolled_into_view
            time.sleep(0.1)

        # This second time the jobs element has all the neccesary information inside of them
        jobs = browser.find_element(By.XPATH,"//ul[contains(@class,'scaffold-layout__list-container')]").find_elements_by_xpath('./*')

        for job in jobs:
            job.click()
            time.sleep(0.3)
            insert_row = {
                        'Job Title':       job.get_attribute('innerText').split('\n')[0],
                        'Job Company':     job.get_attribute('innerText').split('\n')[1],
                        'Job Location':    job.get_attribute('innerText').split('\n')[2],
                        'Link':            job.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'),
                        'Job Description': browser.find_element(By.ID,'job-details').get_attribute('innerText'),
                        'Job Words':       browser.find_element(By.ID,'job-details').get_attribute('innerText').split()
                        }

            df = pd.concat([df, pd.DataFrame([insert_row])])
            
        # Actual page analysis
        for page_number in pagination_elements:
            if page_number.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-current') == 'true':
                current_page = page_number
                #print('The current page is', page_number.get_attribute('data-test-pagination-page-btn'))

            try:
                if int(re.findall(r'\d+',page_number.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]) == int(re.findall(r'\d+',current_page.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]) + 1:
                    page_number.click()
                    break
            except:
                pass
        try:
            if int(re.findall(r'\d+',current_page.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]) == max(page_number_array):
                df = df.reset_index(drop = True)
                df.to_excel(str(linkedin_csv_headers[contador_global]) + '.xlsx')
                break

            if len(pagination_elements) == 0:
                df = df.reset_index(drop = True)
                df.to_excel(str(linkedin_csv_headers[contador_global]) + '.xlsx')
                break
        except:
            pass
    contador_global += 1

browser.close()

The number of jobs of this search is 80
The number of jobs of this search is 246
The number of jobs of this search is 251
The number of jobs of this search is 39
The number of jobs of this search is 122


In [45]:
df_test            = pd.read_excel('Test.xlsx')
df_data_analyst    = pd.read_excel('Data Analyst.xlsx')
df_data_scientist  = pd.read_excel('Data Scientist.xlsx')
df_product_analyst = pd.read_excel('Product Analyst.xlsx')
df_product_owner   = pd.read_excel('Product Owner.xlsx')

display(df_data_analyst)

,Unnamed: 0,Job Title,Job Company,Job Location,Link,Job Description,Job Words
0,0,Senior Data and Web Analyst,EPAM Anywhere,"Kallithea, Egeo Meridional, Grecia",https://www.linkedin.com/jobs/view/3331005885/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
1,1,Data Analyst,Afterpay,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/3305395872/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
2,2,Data Analyst,Averity,Nueva York y alrededores,https://www.linkedin.com/jobs/view/3313359886/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
3,3,Data Analyst - (Remote - US),Mediavine,"Nueva York, NY",https://www.linkedin.com/jobs/view/3335122743/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
4,4,Data analyst,Excelya,Francia,https://www.linkedin.com/jobs/view/3329355728/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
...,...,...,...,...,...,...,...
233,233,Lead Data Scientist (Data Science/Data Analyti...,First Help Financial,"Needham, MA",https://www.linkedin.com/jobs/view/3327321660/...,Programmatic Analyst\n\nExperience - 3+ years\...,"['Programmatic', 'Analyst', 'Experience', '-',..."
234,234,Senior Data Scientist | Inference & Estimation,Ramp,"Nueva York, NY",https://www.linkedin.com/jobs/view/3333719561/...,Programmatic Analyst\n\nExperience - 3+ years\...,"['Programmatic', 'Analyst', 'Experience', '-',..."
235,235,Senior Data Scientist- (Data modeler),Ria Money Transfer,"Región Metropolitana de Santiago, Chile",https://www.linkedin.com/jobs/view/3295859385/...,Programmatic Analyst\n\nExperience - 3+ years\...,"['Programmatic', 'Analyst', 'Experience', '-',..."
236,236,"Data Scientist, Trust & Safety",Wish,"San Francisco, CA",https://www.linkedin.com/jobs/view/3328541247/...,Programmatic Analyst\n\nExperience - 3+ years\...,"['Programmatic', 'Analyst', 'Experience', '-',..."


In [50]:
df_data_analyst = df_data_analyst.reset_index(drop = True)
df_filtered = df_data_analyst[df_data_analyst["Job Description"].str.contains("100%")]
display(df_filtered)


,Unnamed: 0,Job Title,Job Company,Job Location,Link,Job Description,Job Words
0,0,Senior Data and Web Analyst,EPAM Anywhere,"Kallithea, Egeo Meridional, Grecia",https://www.linkedin.com/jobs/view/3331005885/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
1,1,Data Analyst,Afterpay,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/3305395872/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
2,2,Data Analyst,Averity,Nueva York y alrededores,https://www.linkedin.com/jobs/view/3313359886/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
3,3,Data Analyst - (Remote - US),Mediavine,"Nueva York, NY",https://www.linkedin.com/jobs/view/3335122743/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
4,4,Data analyst,Excelya,Francia,https://www.linkedin.com/jobs/view/3329355728/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
...,...,...,...,...,...,...,...
169,169,"Data Analyst, Trilogy (Remote) - $60,000/year USD",Crossover for Work,"Nueva Delhi, Delhi, India",https://www.linkedin.com/jobs/view/3325103836/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
170,170,"Data Research Analyst, Trilogy (Remote) - $60,...",Crossover for Work,Al-Qassim Region,https://www.linkedin.com/jobs/view/3325103847/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
171,171,Electronic Data Interchange Analyst,FTSi.Tech,"Memphis, TN",https://www.linkedin.com/jobs/view/3331701516/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."
172,172,"Data Analyst, Forward Kitchens (Remote) - $60,...",Crossover for Work,"Johannesburgo, Gauteng, Sudáfrica",https://www.linkedin.com/jobs/view/3325109494/...,Are you a tech professional looking for remote...,"['Are', 'you', 'a', 'tech', 'professional', 'l..."


In [49]:
df_filtered.to_excel('Filtered.xlsx')